In [1]:
import pandas as pd 
import numpy as np 
import tensorflow as tf 
import zipfile
from keras.layers import Dense, Reshape
import matplotlib.pyplot as plt
import math
from keras.losses import *
from keras.models import Sequential, Model
from keras.layers import Dense, Reshape
from keras.layers import LSTM, Flatten, Input, Embedding, concatenate, Concatenate, average, maximum, Dropout
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from keras import optimizers
from keras import backend as K
from keras.callbacks import TensorBoard
from sklearn.utils import shuffle


Using TensorFlow backend.


In [2]:
def balanceFraudDataframe(df, p): #p is desired fraud/non-fraud ratio
    dfFraud = df.loc[df['isFraud']==1,:]
    x = len(dfFraud)
    n = len(df)
    numOfTimesToResample = int((p * n - x)/((1-p) * x))
    #print(numOfTimesToResample)
    dfBalanced = df
    for _ in range(numOfTimesToResample):
        dfBalanced = dfBalanced.append(dfFraud)
    dfBalanced = shuffle(dfBalanced)
    return dfBalanced

In [3]:
def findPercentFraud(df):
    return len(df.loc[df.isFraud == 0, :])/len(df)

In [4]:
df = pd.read_pickle("data/dataframe_train_v06.pd")
df_fraud = pd.read_pickle('dfFraud.pd')

In [5]:
df['isFraud'] = df_fraud['isFraud']

In [6]:
#df.isna().sum().sum()

In [7]:
#scaler = MinMaxScaler(feature_range=(0, 1))
scaler = StandardScaler()
scaler.fit(df.drop('isFraud', axis=1))

StandardScaler(copy=True, with_mean=True, with_std=True)

In [8]:
np.random.seed(1234)
choose = np.random.rand(len(df))
dfTrain = df.loc[choose >= .2, :]
dfTest = df.loc[choose < .2, :]

In [9]:
dfTrain.shape

(472515, 717)

In [10]:
assert len(dfTrain) + len(dfTest) == len(df)

In [11]:
print(findPercentFraud(dfTrain))
dfTrain = balanceFraudDataframe(dfTrain, .15)
print(findPercentFraud(dfTrain))

0.964932330190576
0.8730815261620949


In [12]:
#scaler.fit(df.drop('isFraud', axis=1))

In [13]:
print(findPercentFraud(dfTest))

0.9653209065875874


In [14]:
x_train = scaler.transform(dfTrain.drop(["isFraud"], axis=1))
y_train= np.array(dfTrain['isFraud'])

x_test = scaler.transform(dfTest.drop(["isFraud"], axis=1))
y_test = np.array(dfTest['isFraud'])

In [15]:
from sklearn.utils import class_weight

inputs = Input(shape=(716,))
# We stack a deep densely-connected network on top
x = Dense(800, activation='relu', use_bias=True)(inputs)
x = Dropout(.2)(x)
x = Dense(1000,  activation='relu', use_bias=True)(x)
x = Dropout(.2)(x)
#x = Dense(800, activation='relu', use_bias=True)(x)
#x = Dropout(.2)(x)
#x = Dense(600, activation='relu', use_bias=True)(x)
x = Dense(600, activation='relu', use_bias=True)(x)
x = Dropout(.2)(x)
x = Dense(200, activation='relu', use_bias=True)(x)
# And finally we add the main logistic regression layer
output = Dense(1 , activation='sigmoid', name='ANN_output')(x)

adam = optimizers.Adam(lr=0.001)
model = Model(inputs=inputs, outputs = output)

from sklearn.utils.class_weight import compute_class_weight
weights = class_weight.compute_class_weight('balanced',
                                            np.unique(y_train),
                                            y_train)
# weights = [4, 1]
from sklearn.metrics import roc_auc_score

model.compile(loss="binary_crossentropy", optimizer=adam, metrics=['accuracy', ])
model.fit(x_train, y_train, epochs=20, batch_size=1024, verbose=2, class_weight= weights)


Epoch 1/20
 - 32s - loss: 0.2340 - acc: 0.9176
Epoch 2/20
 - 31s - loss: 0.1923 - acc: 0.9329
Epoch 3/20
 - 31s - loss: 0.1711 - acc: 0.9405
Epoch 4/20
 - 31s - loss: 0.1552 - acc: 0.9460
Epoch 5/20
 - 31s - loss: 0.1409 - acc: 0.9508
Epoch 6/20
 - 31s - loss: 0.1293 - acc: 0.9546
Epoch 7/20
 - 31s - loss: 0.1194 - acc: 0.9575
Epoch 8/20
 - 31s - loss: 0.1108 - acc: 0.9604
Epoch 9/20
 - 31s - loss: 0.1031 - acc: 0.9631
Epoch 10/20
 - 30s - loss: 0.0959 - acc: 0.9653
Epoch 11/20
 - 30s - loss: 0.0894 - acc: 0.9678
Epoch 12/20
 - 30s - loss: 0.0843 - acc: 0.9693
Epoch 13/20
 - 31s - loss: 0.0796 - acc: 0.9708
Epoch 14/20
 - 30s - loss: 0.0748 - acc: 0.9726
Epoch 15/20
 - 31s - loss: 0.0717 - acc: 0.9736
Epoch 16/20
 - 31s - loss: 0.0675 - acc: 0.9752
Epoch 17/20
 - 31s - loss: 0.0649 - acc: 0.9760
Epoch 18/20
 - 31s - loss: 0.0625 - acc: 0.9768
Epoch 19/20
 - 31s - loss: 0.0591 - acc: 0.9782
Epoch 20/20
 - 30s - loss: 0.0569 - acc: 0.9789


In [16]:
#model.compile(loss="binary_crossentropy", optimizer=adam, metrics=['accuracy' ])
#model.fit(x_train, y_train, epochs=50, batch_size=1024, verbose=2, )#class_weight= weights)

In [17]:
model.evaluate(x_test, y_test)

118025/118025 [==============================] - 4s 34us/step


[0.082598519538053986, 0.98089387841609499]

In [18]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, model.predict(x_test))

0.9395065134972902

In [20]:
from datetime import datetime
now=datetime.strftime(datetime.now(), '%m_%d_%H_%M')
model.save("model_v06_%s.h5"%now)

In [22]:
df_subm = pd.read_pickle('./data/dataframe_test_v06.pd')
x_sub = scaler.transform(df_subm)
y_pred = model.predict(x_sub)
dfToSubmit = pd.DataFrame(y_pred)
dfToSubmit.columns = ['isFraud']
dfToSubmit.index = df_subm.index
dfToSubmit

,isFraud
TransactionID,
3663549,3.683707e-10
3663550,7.015032e-15
3663551,2.222747e-23
3663552,5.567483e-09
3663553,3.313289e-10
...,...
4170235,6.818913e-16
4170236,3.550236e-10
4170237,1.628579e-17


In [23]:
dfToSubmit.to_csv('./final_submission_%s.csv'%now)

In [ ]:
trainPredict = model.predict(x_train)
testPredict = model.predict(x_test)
# calculate root mean squared error
trainScore = math.sqrt(mean_squared_error(y_train, trainPredict))
#trainScore = numpy.mean(numpy.abs((y_train - trainPredict))/y_train)

print('Train Score: %.2f MAPE' % (trainScore))
testScore = math.sqrt(mean_squared_error(y_test, testPredict))
#testScore =  numpy.mean(numpy.abs((y_test - testPredict))/y_test)

print('Test Score: %.2f MAPE' % (testScore))


